In [5]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os

##################################################################### IMPORTS EVERYTHING
#USER INPUTS HERE!!!!!!
FilePath = "/Users/timmo/Documents/RapfLab/code/blanktest/files" #FILES you want to analyze here
FilePathList = "/Users/timmo/Documents/RapfLab/code/blanktest" #MASSLIST file (what filepath is it in)
FilePathBlank= "/Users/timmo/Documents/RapfLab/code/blanktest/Processed" #BLANKS (filepath where blank files for standard dev are found)
FileType="csv" #sets filetype
ListFile= "IncompCombinedmasslist.csv" #filename of mass list
BlankFile="stdev_blank.csv" #filename of blank file with its stdevs. Should stay constant
SaveLocation="/Users/timmo/Documents/RAPFLAB/code/Processed/" #where the files will save
FileName="_Processed.csv"
################################






#initializing some random stuff
os. chdir(FilePathList) #sets working directory to FilePathList
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
MassNumberNames=[]
#######################################
os. chdir(FilePathList)
CurrFile = pd.read_csv(ListFile) #reads the file masslist.csv from the filepath --> since we only want to rea

for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the colum mz_list
    MassNumberNames.append(CurrFile['Species'][i])

os. chdir(FilePathBlank)   
Blanks=[]
CurrFile = pd.read_csv(BlankFile)
for i in (range(len(CurrFile))):
    Blanks.append(CurrFile['standard deviation'][i])

    
os. chdir(FilePath) #updates file path to the folder with actual files for processing
AllFiles=pd.DataFrame()
FileType="csv" #what filetype are the files. Will probably be csv
files = sorted(glob.glob("*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in
OutFolder = "Processed" #folder where processed files will be saved
outname = "" #part of file name. Will be used later
##############################

ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
mzlistRep=[]



for entry in range(num_files): #goes through all files
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=1) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
    NameList=(str(FullSplit[0]))
    mzlistRep=[]
    ProcessedX=[] #values for X(Thompsons) will be saved here
    ProcessedY=[] #values for Y(Counts) will be saved here
    PPM=[] #PPM values will be saved here
    names=[]
    BlankRep=[]
    Detected=[]
    #gets the third part of the name (0,1,[2]). Adds it to the name list
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns="#Point") #drops point column
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high
        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)
                    mzlistRep.append(MassNumbersList[mass])
                    names.append(MassNumberNames[mass])
                    BlankRep.append(Blanks[mass])

                  
            
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                                  # number of values no matter what
                ProcessedX.append(0)  
                ProcessedY.append(0) #appends 0 for both 
                PPM.append(0)
                mzlistRep.append(MassNumbersList[mass])
                names.append(MassNumberNames[mass])  
                BlankRep.append(Blanks[mass])

    for i in range(len(ProcessedY)):
        if ProcessedY[i]>=(3*(BlankRep[i])):
              Detected.append("Detected")
        elif ProcessedY[i]<(3*(BlankRep[i])):
               Detected.append("Not Detected")
        else:
            continue

 
    IndFile = pd.DataFrame(columns = ["Compound Names","mz_list",NameList+"_mass",NameList+"_count",NameList+"_PPM", "Not Detected"]) #columns mz_list, mass, and count +NameList[i]
    IndFile["Compound Names"]=names
    IndFile["mz_list"]=mzlistRep
    IndFile[NameList+"_mass"]=ProcessedX
    IndFile[NameList+"_count"]=ProcessedY #for this and the line before, it reads the values of processed from i*length
    IndFile[NameList+"_PPM"]=PPM
    IndFile["Blank STDev"]=BlankRep
    IndFile["Not Detected"]=Detected
    IndFile.to_csv(SaveLocation+NameList+FileName)        

